# Bedrock에서 Claude 3 Haiku 모델 미세 조정
이 노트북은 Amazon Bedrock에서 Claude 3 Haiku 모델을 미세 조정하는 과정을 안내합니다.

## 준비물
- Bedrock에 접근할 수 있는 AWS 계정
- 데이터 세트 (본 노트북에서 제공하는 샘플 데이터 세트를 사용할 수도 있습니다)
- [학습 데이터가 저장된 S3 버킷에 접근 가능한 서비스 역할](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html)

## 의존성 패키지 설치

In [ ]:
!pip install boto3

In [ ]:
import boto3 

## 데이터 세트 준비
Bedrock 미세 조정을 위한 데이터 세트는 JSONL 형식이어야 합니다 (즉, 각 줄이 JSON 객체인 파일입니다).

JSONL 파일의 각 줄은 다음 구조를 따르는 JSON 객체여야 합니다.

```
{
  "system": "<시스템_메시지 (선택 사항)>",
  "messages": [
    {"role": "user", "content": "사용자 메시지 내용"},
    {"role": "assistant", "content": "어시스턴트 응답 내용"},
    ...
  ]
}
```

- `system` 필드는 선택 사항입니다.
- 메시지는 최소 두 개 이상이어야 합니다.
- 첫 번째 메시지의 역할은 "user"여야 합니다.
- 마지막 메시지의 역할은 "assistant"여야 합니다.
- 사용자 메시지와 어시스턴트 메시지는 번갈아 가며 나타나야 합니다.
- 명시된 키 외에 다른 키는 사용할 수 없습니다.


## 샘플 데이터 세트 - JSON 모드
모델이 모든 질문에 JSON 형식으로 응답하도록 학습시키는 샘플 데이터 세트를 포함했습니다. 해당 데이터 세트의 예는 다음과 같습니다.

In [ ]:
import json

sample_dataset = []
dataset_path = 'datasets/json_mode_dataset.jsonl'
with open(dataset_path, 'r') as f:
    for line in f:
        sample_dataset.append(json.loads(line))

print(json.dumps(sample_dataset[0], indent=2))

## S3에 데이터 세트 업로드
미세 조정용 데이터 세트는 S3에서 접근 가능해야 합니다. 이 데모에서는 사용자가 관리하는 S3 버킷에 샘플 데이터 세트를 업로드하여 사용합니다.

In [ ]:
bucket_name = "YOUR_BUCKET_NAME" # TODO: 실제 사용자 버킷 이름을 입력하세요.
s3_path = "json_mode_dataset.jsonl"

s3 = boto3.client('s3')
s3.upload_file(dataset_path, bucket_name, s3_path)

## Bedrock 미세 조정 작업 시작

이제 데이터 세트가 준비되었으니, `boto3`를 사용하여 미세 조정 작업을 시작할 수 있습니다. 먼저 작업 실행에 필요한 몇 가지 파라미터를 설정합니다.

In [ ]:
# 작업 구성
job_name = "anthropic-finetuning-cookbook-training"
custom_model_name = "anthropic_finetuning_cookbook"
role = "YOUR_AWS_SERVICE_ROLE_ARN" # TODO: 실제 AWS 서비스 역할 ARN을 입력하세요.
output_path = f"s3://{bucket_name}/finetuning_example_results/"
base_model_id = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0:200k"

# 하이퍼파라미터 설정
epoch_count = 5
batch_size = 4
learning_rate_multiplier = 1.0

그다음, `boto3`를 사용하여 미세 조정 작업을 시작합니다.

In [ ]:
bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

bedrock.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": f"{epoch_count}",
        "batchSize": f"{batch_size}",
        "learningRateMultiplier": f"{learning_rate_multiplier}",
    },
    trainingDataConfig={"s3Uri": f"s3://{bucket_name}/{s3_path}"},
    outputDataConfig={"s3Uri": output_path},
)

다음 코드를 사용하여 학습이 진행 중인 작업의 상태를 확인할 수 있습니다.

In [ ]:
# 미세 조정 작업 상태 확인
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]

## 미세 조정된 모델 사용하기!

미세 조정된 모델을 사용하려면, [Amazon Bedrock의 프로비저닝된 처리량(Provisioned Throughput)을 사용하여 모델을 호스팅해야 합니다](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-use.html). 프로비저닝된 처리량을 통해 모델이 준비되면, Bedrock API를 사용하여 모델을 호출할 수 있습니다.

In [ ]:
provisioned_throughput_arn = "YOUR_PROVISIONED_THROUGHPUT_ARN" # TODO: 실제 프로비저닝된 처리량 ARN을 입력하세요.

In [ ]:
bedrock = boto3.client('bedrock-runtime', region_name = "us-east-1")
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "system": "JSON 모드: 활성화됨 (미세 조정 데이터셋의 시스템 프롬프트와 일치시킵니다)",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":"대규모 언어 모델(LLM)이란 무엇인가요?"
                    }
                ]
            }
        ]
    }
)
response = bedrock_runtime.invoke_model(
	modelId=provisioned_throughput_arn,
    body=body
)
body = json.loads(response['body'].read().decode('utf-8'))

In [ ]:
print(body['content'][0]['text'])